# 使用 XPath 解析网页

XPath 是一种在 XML/HTML 文档中定位节点的语言, 在爬虫中常用于精准提取数据。本文示例主要基于 lxml 与 parsel 两个库。

## 安装依赖
```shell
pip install lxml parsel chardet
```
- lxml: 高性能 HTML/XML 解析器
- parsel: 对 XPath/CSS 进一步封装, 与 scrapy 中 Selector 一致
- chardet: 可选, 用于编码检测

## 一段示例 HTML
我们以一段简化的 HTML 作为练习:

In [4]:
html = '''\n<html>\n  <head>\n    <title>示例页面 - 测试</title>\n  </head>\n  <body>\n    <div id="container" class="main wrap">\n      <h1 class="title">主标题<span>副标题</span></h1>\n      <ul class="items">\n        <li data-id="1001"><a href="/detail/1001">苹果</a></li>\n        <li data-id="1002"><a href="/detail/1002">香蕉</a></li>\n        <li data-id="1003"><a href="/detail/1003">菠萝</a></li>\n      </ul>\n      <p class="desc">  这是 一个 带有   多余空格 的 描述。 </p>\n      <div class="links">\n        <a href="https://example.com/about" class="nav">关于我们</a>\n        <a href="https://example.com/contact" class="nav external">联系我们</a>\n      </div>\n    </div>\n  </body>\n</html>\n'''

## 使用 lxml.etree 加载与 XPath 查询
lxml 的 html.fromstring 能自动容错解析非严格 HTML。

In [5]:
from lxml import html as lxml_html
tree = lxml_html.fromstring(html)  # 构建 DOM 树

# 1. 选取 <title> 文本
title_text = tree.xpath('//title/text()')[0]
print('页面标题:', title_text)

# 2. 选取所有商品名称 (li 下 a 文本)
items = tree.xpath('//ul[@class="items"]/li/a/text()')
print('商品列表:', items)

# 3. 获取第二个 li 的 data-id (位置选择)
second_id = tree.xpath('//ul[@class="items"]/li[2]/@data-id')[0]
print('第二个商品ID:', second_id)

# 4. 获取 class 包含 external 的链接 href
external_href = tree.xpath('//a[contains(@class, "external")]/@href')[0]
print('external链接:', external_href)

# 5. 使用 normalize-space 去除多余空格
desc = tree.xpath('normalize-space(//p[@class="desc"])')
print('描述(去空格):', desc)

# 6. 选取副标题 span 文本
subtitle = tree.xpath('//h1[@class="title"]/span/text()')[0]
print('副标题:', subtitle)

页面标题: 示例页面 - 测试
商品列表: ['苹果', '香蕉', '菠萝']
第二个商品ID: 1002
external链接: https://example.com/contact
描述(去空格): 这是 一个 带有 多余空格 的 描述。
副标题: 副标题


## 常见 XPath 语法速查
| 场景 | XPath 示例 | 说明 |
|------|------------|------|
| 根节点匹配 | `/html/body` | 从根逐层 |
| 任意层级 | `//div` | 匹配所有 div |
| 属性过滤 | `//ul[@class='items']` | 精确属性 |
| 包含属性 | `//a[contains(@class,'nav')]` | 模糊 class |
| 多条件 AND | `//a[@class='nav'][contains(@href,'contact')]` | 链式条件 |
| 取文本 | `//h1/text()` | 当前节点直接文本 |
| 所有后代文本 | `//h1//text()` | 包含子孙 |
| 属性值 | `//li/@data-id` | 取属性 |
| 位置 | `//li[1]` | 第一    (从1开始) |
| 最后一个 | `//li[last()]` | 尾元素 |
| 条件函数 | `//a[starts-with(@href,'https://')]` | 前缀判断 |
| 计数 | `count(//li)` | 统计节点数 |
| 去空格 | `normalize-space(//p)` | 清理空白 |
| 上级/父节点 | `//span/..` | 返回父节点 |
| 轴: following-sibling | `//li[@data-id='1001']/following-sibling::li` | 后面同级 |
| 轴: preceding-sibling | `//li[@data-id='1003']/preceding-sibling::li` | 前面同级 |

## 抓取真实网页示例
以 httpbin.org 为例(演示结构解析), 实际生产中需处理编码与容错。

In [6]:
import requests, chardet
url = 'https://httpbin.org/html'  # 返回一段简单 HTML
resp = requests.get(url, timeout=10)
# 编码检测 (有些站点未正确声明)
raw = resp.content
enc = chardet.detect(raw)['encoding'] or resp.apparent_encoding or 'utf-8'
text = raw.decode(enc, errors='ignore')
doc = lxml_html.fromstring(text)

# 提取所有链接与段落
links = doc.xpath('//a/@href')
paras = [p.strip() for p in doc.xpath('//p//text()') if p.strip()]
print('链接:', links)
print('段落:', paras[:3], '...')
# 安全获取节点文本的封装函数
def xtext(node, path, default=None):
    try:
        r = node.xpath(path)
        return r[0] if r else default
    except Exception:
        return default

print('示例 title:', xtext(doc, '//title/text()', 'N/A'))

链接: []
段落: ["Availing himself of the mild, summer-cool weather that now reigned in these latitudes, and in preparation for the peculiarly active pursuits shortly to be anticipated, Perth, the begrimed, blistered old blacksmith, had not removed his portable forge to the hold again, after concluding his contributory work for Ahab's leg, but still retained it on deck, fast lashed to ringbolts by the foremast; being now almost incessantly invoked by the headsmen, and harpooneers, and bowsmen to do some little job for them; altering, or repairing, or new shaping their various weapons and boat furniture. Often he would be surrounded by an eager circle, all waiting to be served; holding boat-spades, pike-heads, harpoons, and lances, and jealously watching his every sooty movement, as he toiled. Nevertheless, this old man's was a patient hammer wielded by a patient arm. No murmur, no impatience, no petulance did come from him. Silent, slow, and solemn; bowing over still further his chronically

## 使用 parsel.Selector
parsel 提供更链式的 API, 支持 XPath 与 CSS 混用。

In [7]:
from parsel import Selector
sel = Selector(text=html)
# XPath
titles = sel.xpath('//ul[@class="items"]/li/a/text()').getall()
print('parsel 商品:', titles)
# get() 只取第一个, getall() 取全部
first_href = sel.xpath('//ul[@class="items"]/li[1]/a/@href').get()
print('第一个href:', first_href)
# CSS 选择器混用
nav_texts = sel.css('div.links a.nav::text').getall()
print('导航文本:', nav_texts)
# 链式过滤: 先选节点再取属性
hrefs_chain = sel.css('ul.items li a').xpath('./@href').getall()
print('链式href:', hrefs_chain)

parsel 商品: ['苹果', '香蕉', '菠萝']
第一个href: /detail/1001
导航文本: ['关于我们', '联系我们']
链式href: ['/detail/1001', '/detail/1002', '/detail/1003']


## 复杂条件与常见模式
1. 根据多个属性: `//a[@class='nav' and contains(@href,'contact')]`
2. 排除某些节点: `//a[not(contains(@class,'external'))]`
3. 选择第 2~最后一个节点: `//li[position()>1]`
4. 选择文本包含某关键词: `//li[a[contains(text(),'果')]]`
5. 嵌套取属性后再跳回父节点: `//a[contains(@href,'detail')]/../@data-id`
6. 相对路径区别: `./` 表示从当前节点开始, `//` 表示任意后代。

## 常见坑与技巧
- HTML 不规范: 优先使用 `lxml.html.fromstring`, 少用纯 xml 解析。
- 多余空白: 使用 `normalize-space()` 或 Python `strip()`。
- class 多值匹配: 不要用等号硬匹配时可用 `contains(@class,'name')`。
- 性能: 合并 XPath 查询, 避免在循环中反复 `xpath()` 大量小查询。
- 容错: 写辅助函数安全取值(见上 xtext)。
- 调试: 输出 `etree.tostring(node, pretty_print=True).decode()` 查看结构。
- 编码: 使用 `resp.content` + chardet 检测, 避免乱码影响匹配。
- 动态页面: 需结合 Selenium 或请求其 AJAX 接口, XPath 只解析已有 HTML。

## 总结
1. XPath 用于精准定位, 与正则相比更稳定。
2. 熟练掌握节点定位、条件过滤、文本与属性提取是关键。
3. parsel 提升易用性, 在 scrapy 中几乎同样用法。
4. 遇到复杂结构先 prettify/打印节点, 再逐步收窄 XPath。

## 详细分解复杂 XPath 表达式


### 获取 class 包含 j_readContent 的 div 内所有文本节点
`//div[contains(concat(' ', normalize-space(@class), ' '), ' j_readContent ')]//text()`

组成部分说明:
\- `//div`：从文档根部选取所有 `div` 元素 (任意层级后代)。
\- `[ ... ]`：谓词过滤，只保留满足条件的 `div`。
\- `@class`：获取该元素的 `class` 属性字符串。
\- `normalize-space(@class)`：去掉首尾空白并把中间连续空白压缩为单个空格，标准化类名列表。
\- `concat(' ', normalize-space(@class), ' ')`：在标准化后的类串前后各拼接一个空格，形成模式 `" class1 class2 class3 "`，为后续“整词匹配”制造边界。
\- `contains( ..., ' j_readContent ')`：检查是否包含带前后空格的子串 `' j_readContent '`，只在独立类名存在时匹配，避免误匹配 `j_readContentExtra`、`my_j_readContent_old` 等。
\- `//text()`：在匹配的 `div` 内继续向下选取所有后代文本节点（包括嵌套标签中的文本）。

为何不直接用 `//div[contains(@class,'j_readContent')]//text()`:
\- 该写法会匹配任何包含该子串的类值，易误伤类似 `j_readContentExtra`。
\- 使用空格包裹后通过“整词”判定，确保只匹配独立的类名 `j_readContent`。

要只取该 `div` 直接子级文本可改为:
`//div[contains(concat(' ', normalize-space(@class), ' '), ' j_readContent ')]/text()`

示例提取并清洗:
```python
from parsel import Selector

def extract_content(html: str):
    sel = Selector(html)
    texts = sel.xpath("//div[contains(concat(' ', normalize-space(@class), ' '), ' j_readContent ')]//text()").getall()
    return "\n".join(t.strip() for t in texts if t.strip())
```

要匹配多个类名同时存在(例如同时包含 `j_readContent` 与 `active`):
```python
xpath = ("//div"
         "[contains(concat(' ', normalize-space(@class), ' '), ' j_readContent ')"
         " and contains(concat(' ', normalize-space(@class), ' '), ' active ')]")
```